In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist
from keras.layers import Dense, Input, Activation, Dropout
from keras.models import Model
from keras.models import Sequential

In [ ]:
# Carregando o conjunto de dados MNIST
(x_train, y_train), (x_test, y_test) = (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

#yn = y_train.to_numpy()
print(y_train.max())

# Normalizando os valores dos pixels para o intervalo [0, 1]
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# Transformando as imagens em vetores unidimensionais (784 pixels)
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

# Aplicação de ruído nos dados e armazenando e variáveis acessórias
noise_factor = 0.3
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)

#print(len(x_train))
#print(x_train.shape[1:])
#print(np.prod(x_train.shape[1:]))
print(x_train.shape)
print(x_test.shape)

#print(x_train.view)
print(y_train)

In [ ]:
# Definindo a dimensão da representação codificada
n_latent = 120

# Criando o modelo do autoencoder
model = Sequential()
model.add(Dense(128, input_shape=(784,), activation="relu"))
model.add(Dense(300, activation="relu"))
model.add(Dense(n_latent, activation="relu"))
model.add(Dense(300, activation="relu"))
model.add(Dense(784, activation="sigmoid"))

# Compilando o modelo
model.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
# Treinando o autoencoder
model.fit(x_train, x_train, epochs=30, batch_size=256, shuffle=True, validation_data=(x_test, x_test))

# Codificando e decodificando as imagens de teste sem ruído
encoded_imgs = model.predict(x_test)

In [ ]:
# Visualizando algumas imagens originais sem ruído e reconstruídas
n = 15
plt.figure(figsize=(20, 4))
for i in range(n):
    # Imagem original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Imagem reconstruída
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(encoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.show()

In [ ]:
# Codificando e decodificando as imagens de teste com ruído
encoded_imgs = model.predict(x_test_noisy)

In [ ]:
# Visualizando algumas imagens originais com ruído e reconstruídas
n = 15
plt.figure(figsize=(20, 4))
for i in range(n):
    # Imagem original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test_noisy[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Imagem reconstruída
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(encoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.show()

In [ ]:
batch_size = 128
hidden_units = 256
dropout = 0.45
input_size = x_train.shape[1]
num_labels = len(np.unique(y_train))

# model is a 3-layer MLP with ReLU and dropout after each layer
model2 = Sequential()
model2.add(Dense(hidden_units, input_dim=input_size))
model2.add(Activation('relu'))
#model2.add(Dropout(dropout))
model2.add(Dense(hidden_units))
model2.add(Activation('relu'))
#model2.add(Dropout(dropout))
model2.add(Dense(num_labels))
model2.add(Activation('softmax'))

model2.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model2.fit(x_train, y_train, epochs=50, batch_size=batch_size)

In [ ]:
loss, acc = model2.evaluate(x_test, y_test, batch_size=batch_size)
print("\nTeste de Acurácia Classificação sem Ruído: %.1f%%" % (100.0 * acc))
loss, acc = model2.evaluate(x_test_noisy, y_test, batch_size=batch_size)
print("\nTeste de Acurácia Classificação com Ruído: %.1f%%" % (100.0 * acc))
loss, acc = model2.evaluate(encoded_imgs, y_test, batch_size=batch_size)
print("\nTeste de Acurácia Classificação das Imagens Recuperadas do Ruído: %.1f%%" % (100.0 * acc))